In [146]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import ast
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.metrics import mean_squared_error

In [147]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)

    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

def flatten_hits(df):
    df_ = pd.DataFrame()
    for index, row in df.iterrows():
        initial_id = df['fullVisitorId'][index]
        s = json.dumps(df['hits'][index])
        r = json.loads(s)
        d = ast.literal_eval(r)
        for each in d:
            each['fullVisitorId'] = initial_id
        column_as_df = json_normalize(d)
        if 'product' in column_as_df.columns:
            column_as_df['v2ProductName'] = column_as_df['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            column_as_df['v2ProductCategory'] = column_as_df['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            del column_as_df['product']
        if 'promotion' in column_as_df.columns:
            column_as_df['promoId']  = column_as_df['promotion'].apply(lambda x: [p['promoId'] for p in x] if type(x) == list else [])
            column_as_df['promoName']  = column_as_df['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            del column_as_df['promotion']
        df_ = df_.append(column_as_df)
    df = df.merge(df_, on='fullVisitorId')
    return df


In [148]:
### Loading TRAIN Data
df_train = load_df('../ga-customer-revenue-prediction/train_v2.csv', nrows=450)
# for each in df_train.columns:
#     print(each)
# print('-'*20)
df_train = flatten_hits(df_train)
# for each in df_train.columns:
#     print(each)
### Loading TEST Data
df_test = load_df('../ga-customer-revenue-prediction/test_v2.csv', nrows=450)
df_test = flatten_hits(df_test)


Loaded train_v2.csv. Shape: (450, 59)


/Users/Chris/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Loaded test_v2.csv. Shape: (450, 58)


In [149]:
ones = []
for each in df_train.columns:
#     print(str(each) + ': ' + str(df_train[each].nunique()))
    try:
        if df_train[each].nunique() == 1:
            ones.append(each)
    except:
        print(each)
cols_to_remove = [x for x in ones if set(df_train[x].unique()) == set(['not available in demo dataset'])]
cols_to_remove.append('hits')
cols_to_remove.append('customDimensions_x')
cols_to_remove.append('customDimensions_y')

cols_to_remove.append('customVariables')
cols_to_remove.append('customMetrics')
cols_to_remove.append('experiment')
cols_to_remove.append('promoId')
cols_to_remove.append('promoName')
cols_to_remove.append('promotionActionInfo.promoIsView')
cols_to_remove.append('publisher_infos')
cols_to_remove.append('v2ProductCategory')
cols_to_remove.append('v2ProductName')

customDimensions_y
customMetrics
customVariables
experiment
promoId
promoName
publisher_infos
v2ProductCategory
v2ProductName


In [150]:
#### CHANGED THIS TO GET AVERAGE ####
# average = df_train['totals.totalTransactionRevenue'].dropna().mean()

y = df_train['totals.totalTransactionRevenue'].fillna(0).astype(float)
y = y.apply(lambda x: np.log1p(x))
df_train = df_train.drop('totals.totalTransactionRevenue', axis=1)


In [151]:
### Removing columns that contain no data
cols_to_remove_train = list(cols_to_remove)
cols_to_remove_train.append('page.searchCategory')
cols_to_remove_train.append('page.searchKeyword')
cols_to_remove_train.append('totals.bounces')
df_train = df_train.drop(list(cols_to_remove_train), axis=1)
df_test = df_test.drop(list(cols_to_remove), axis=1)

In [152]:
# pd.set_option('display.max_rows', -1)
# df_train.iloc[432]
for col in df_test.columns:
    print(col)
    print(df_test[col].unique())
    print('-'*20)

channelGrouping
['Organic Search' 'Direct' 'Affiliates' 'Referral' 'Paid Search']
--------------------
date
[20180511]
--------------------
fullVisitorId
['7460955084541987166' '460252456180441002' '3461808543879602873'
 '975129477712150630' '8381672768065729990' '2866297766347322467'
 '2235365487897339889' '1303090465617023038' '0459669224143241747'
 '589440789980446309' '5299820794845174508' '7414906446453226824'
 '4063440620410560213' '3326540121689239742' '7713658643915819491'
 '7437323504893211921' '9553181489247767015' '4605774125139978152'
 '2400060996995525389' '3524623036868660621' '8935547912864718155'
 '6535191149463473748' '3099959025206912558' '7731199528001457558'
 '6228341191093238277' '1237026169588168141' '7075733582376632838'
 '713763283771900341' '8013368769702493385' '3002134698450238670'
 '0749263656297205552' '9955654833335971589' '8598143869506191067'
 '0279221615720490607' '2812481666998392163' '4841488230244428362'
 '1095266983722783832' '5379615238413763276' '

In [153]:
cat_columns = ['channelGrouping',
               'socialEngagementType',
               'device.browser',
               'device.deviceCategory',
               'device.operatingSystem',
               'geoNetwork.city',
               'geoNetwork.continent',
               'geoNetwork.country',
               'geoNetwork.metro', 
               'geoNetwork.networkDomain',
               'geoNetwork.region', 
               'geoNetwork.subContinent',
               'trafficSource.adContent',
               'trafficSource.adwordsClickInfo.adNetworkType',
               'trafficSource.adwordsClickInfo.gclId',
               'trafficSource.adwordsClickInfo.page',
               'trafficSource.adwordsClickInfo.slot', 
               'trafficSource.campaign',
               'trafficSource.keyword',
               'trafficSource.referralPath',
               'trafficSource.source',
               'trafficSource.medium',
               'appInfo.exitScreenName',
               'appInfo.landingScreenName',
               'appInfo.screenName',
               'contentGroup.contentGroup1',
               'contentGroup.contentGroup2',
               'contentGroup.contentGroup3',
               'contentGroup.contentGroup4',
               'contentGroup.contentGroup5',
               'contentGroup.previousContentGroup1',
               'contentGroup.previousContentGroup2',
               'contentGroup.previousContentGroup3',
               'contentGroup.previousContentGroup4',
               'contentGroup.previousContentGroup5',
               'dataSource',
               'item.currencyCode',
               'item.transactionId',
               'transaction.currencyCode',
               'page.hostname',
               'page.pagePath',
               'page.pagePathLevel1',
               'page.pagePathLevel2',
               'page.pagePathLevel3',
               'page.pagePathLevel4',
               'page.pageTitle',
#                'page.searchCategory',
#                'page.searchKeyword',
               'referer',
               'social.socialNetwork',
               'social.socialInteractionNetworkAction',
               'social.hasSocialSourceReferral',
               'type',
               'transaction.transactionId',
               'transaction.affiliation',
               'eventInfo.eventLabel',
               'eventInfo.eventCategory',
               'eventInfo.eventAction',
               'eCommerceAction.option'
              ]


In [154]:
from sklearn import preprocessing
for each in cat_columns:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train[each].values) + list(df_test[each].values))
    df_train[each] = lbl.transform(list(df_train[each].values))
    df_test[each] = lbl.transform(list(df_test[each].values))

In [155]:
# Getting our vailidation y for scoring
y_true = df_test['totals.totalTransactionRevenue'].fillna(0).astype(float)
y_true = y_true.apply(lambda x: np.log1p(x))
df_test = df_test.drop('totals.totalTransactionRevenue', axis=1)

y_mean = np.mean(y)
y_base = np.full_like(y_true, y_mean)

In [156]:
# df_train = df_train.drop('customDimensions', axis=1)
# df_test = df_test.drop('customDimensions', axis=1)
df_train = df_train.drop('totals.transactionRevenue', axis=1)
df_test = df_test.drop('totals.transactionRevenue', axis=1)
df_train = df_train.drop('totals.transactions', axis=1)
df_test = df_test.drop('totals.transactions', axis=1)

In [157]:
def preprocess(df):
#     df['totals.bounces'] = df['totals.bounces'].fillna(0).astype(np.float)
    df['totals.newVisits'] = df['totals.newVisits'].fillna(0).astype(np.float)
    df['totals.timeOnSite'] = df['totals.newVisits'].fillna(0).astype(np.float)
#     df['totals.transactionRevenue'] = df['totals.transactionRevenue'].fillna(0).astype(np.float)
#     df['totals.transactions'] = df['totals.transactions'].fillna(0).astype(np.float)
    df['trafficSource.adwordsClickInfo.isVideoAd'] = df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(0).astype(np.float)
    df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].fillna(0).astype(np.float)
    
    return df
df_train = preprocess(df_train)
df_test = preprocess(df_test)

In [158]:
#### IMPORTANT ####
# Find the issue here before proceeding on Flux

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [159]:
from sklearn import tree
clf_tree = tree.DecisionTreeRegressor()
clf_tree = clf_tree.fit(df_train, y)

In [160]:
# for col in df_train.columns:
#     if col not in df_test.columns:
#         print(col)

In [161]:
y_pred = clf_tree.predict(df_test)

In [162]:
from math import sqrt
RMSE = sqrt(mean_squared_error(y_true, y_pred))

In [163]:
print(RMSE)

3.9822501303830666


In [164]:
for idx, each in enumerate(clf_tree.feature_importances_):
    print(idx, each*1e5)

print('-'*10)

for idx, each in enumerate(df_train.columns):
    print(idx, each)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 66896.7280163599
19 0.0
20 0.0
21 33103.2719836401
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 0.0
64 0.0
65 0.0
66 0.0
67 0.0
68 0.0
69 0.0
70 0.0
71 0.0
72 0.0
73 0.0
74 0.0
75 0.0
76 0.0
77 0.0
78 0.0
79 0.0
80 0.0
81 0.0
82 0.0
83 0.0
84 0.0
85 0.0
86 0.0
87 0.0
88 0.0
89 0.0
90 0.0
91 0.0
92 0.0
93 0.0
94 0.0
95 0.0
96 0.0
97 0.0
98 0.0
99 0.0
100 0.0
101 0.0
----------
0 channelGrouping
1 date
2 fullVisitorId
3 socialEngagementType
4 visitId
5 visitNumber
6 visitStartTime
7 device.browser
8 device.deviceCategory
9 device.isMobile
10 device.operatingSystem
11 geoNetwork.city
12 geoNetwork.continent
13 geoNetwork.country
14

In [165]:
y_pred = clf_tree.predict(df_test)
y_pred_train = clf_tree.predict(df_train)

RMSE_test = sqrt(mean_squared_error(y_true, y_pred))
RMSE_train = sqrt(mean_squared_error(y, y_pred_train))

print('TEST RMSE:'+str(RMSE_test))
print('TRAIN RMSE:'+str(RMSE_train))


TEST RMSE:3.9822501303830666
TRAIN RMSE:2.0834333010446078e-16


In [ ]:
5.061783422241361